In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import experimenter
import pandas as pd 
import matplotlib.pyplot as plt 

sns.set_theme()

In [ ]:
# Maximum number of replicas that the system can manage.
MAX_REPLICA_COUNT = 100

# Number of experiments
NUMBER_OF_EXPERIMENTS = 5

# Each experiment will be done by this many times for consistency.
EXPERIMENT_REPITITION = 5 

# Int array of replicas for each experiment
REPLICAS: np.ndarray = experimenter.calculate_number_of_replicas(MAX_REPLICA_COUNT, NUMBER_OF_EXPERIMENTS)

# How much should we wait for the kubernetes api to be stabilized
COOLDOWNS: np.ndarray = 2 * np.ones((NUMBER_OF_EXPERIMENTS,), dtype=int) + (REPLICAS * 0.004).astype(int) # Make it ~3 mins

# Namespace of the experimentation
NAMESPACE = "experiments"

# Driver used in kubectl
DRIVER = "minikube" # minikube, kwok, native

# Framework
FRAMEWORK = "native" # native, edgenet (Nisan ortasi) | liqo, kubefed (Sonradan)
# EdgeNet scalability test, how many compute cluster can be tested...

In [ ]:
def draw_replicas(replicas=REPLICAS):
    plt.figure(figsize=(8, 4))
    plt.plot(range(len(replicas)), replicas)
    plt.title("Replica Count by Experiments")
    plt.xlabel("Experiment")
    plt.ylabel("Replicas")
    plt.show()

draw_replicas()
print(COOLDOWNS)
print(REPLICAS)

In [4]:
# Do the experiment
# experimenter.experiment_with(FRAMEWORK, DRIVER, 1, [1, 1], [1, 1])
# experimenter.experiment_with(FRAMEWORK, DRIVER, EXPERIMENT_REPITITION, REPLICAS.tolist(), COOLDOWNS.tolist())

In [7]:
KNATIVE_EXPERIMENT_NAME = "exps/20230310_054850.feather"

def process_experiment(filename):
    data_df = pd.read_feather(filename)

    data_df["api_server_creation_delta"] = data_df["event_created_at"] - data_df["before_create_timestamp"]
    data_df["api_server_deletion_delta"] = data_df["event_deleted_at"] - data_df["before_delete_timestamp"]

    # ADD OTHER TIME DELTA CALCULATIONS HERE
    
    data_df.to_feather(filename)
    return data_df

knative_minikube_df = process_experiment(KNATIVE_EXPERIMENT_NAME)
# knative_minikube_df.columns
knative_minikube_df.columns

Index(['before_create_timestamp', 'before_firstwait_timestamp',
       'before_delete_timestamp', 'before_secondwait_timestamp',
       'before_cooldown_timestamp', 'before_join_timestamp', 'replicas',
       'framework', 'error_occured', 'errors_rised', 'step', 'total_steps',
       'cool_period', 'driver', 'command', 'event_created_at',
       'event_deleted_at', 'event_modified_at', 'event_replicas_at',
       'first_created_at', 'last_created_at', 'first_deleted_at',
       'last_deleted_at', 'first_scheduled_at', 'last_scheduled_at',
       'api_server_creation_delta', 'api_server_deletion_delta'],
      dtype='object')

In [8]:
knative_minikube_df.first_created_at

0   2023-03-10 05:48:51.539694
Name: first_created_at, dtype: datetime64[ns]

In [ ]:
# knative_minikube_df.to_feather("./saved_experiments/native_minikube_20230228_011357.feather")

In [ ]:
# BETTER PROCESS THIS DATA
plt.figure(figsize=(20, 4))
plt.subplot(1, 2, 1)
plt.plot(knative_minikube_df["api_server_deletion_delta"]); plt.title("Creation Times")
plt.subplot(1, 2, 2)
plt.plot(knative_minikube_df["api_server_creation_delta"]); plt.title("Deletion Times")
plt.show()